<a href="https://colab.research.google.com/github/aadyakoirala/analytics-projects/blob/main/Revenue_prediction_coupon_strategies_ipnyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Revenue Prediction Coupon Strategies

In [ ]:
## Preliminaries: packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf


In [ ]:
# load the data
data = pd.read_excel('521704-XLS-ENG.xlsx', sheet_name='AB_test_demog')
print(data.columns)
print(data.head())
print(data.describe())




Index(['id', 'trans_after', 'revenue_after', 'test_coupon', 'minority',
       'non_male', 'channel_acq', 'num_past_purch', 'spent_last_purchase',
       'weeks_since_visit', 'browsing_minutes', 'shopping_cart'],
      dtype='object')
     id  trans_after  revenue_after  test_coupon  minority  non_male  \
0  AB_1            0            0.0            0         0         0   
1  AB_2            0            0.0            1         0         0   
2  AB_3            0            0.0            1         0         0   
3  AB_4            0            0.0            0         1         1   
4  AB_5            0            0.0            1         0         1   

   channel_acq  num_past_purch  spent_last_purchase  weeks_since_visit  \
0            2               6                62.99                  6   
1            1               2                53.99                  0   
2            2               3                88.98                  3   
3            2               1      

**Question 1.** How many rows and columns are in the data? What are all the pre-experiment variables?


In [ ]:
data.shape

(5000, 12)

The pre-experiment variables are minority, non_male, channel_acq, num_past_purch, spent_last_purchase, weeks_since_visit, browsing_minutes, shopping_cart. There are 5000 rows as customers and 12 columns as 12 variables.



In [ ]:
# Regression analysis of ATEs

## Unadjusted ATE(Uncontrolled)
formula1 = 'revenue_after ~ C(test_coupon)'
model1 = smf.ols(formula=formula1, data=data).fit()
model1 = model1.get_robustcov_results(cov_type='HC3')
print(model1.summary())


## Adjusted ATE with Covariates(Controlled)

formula2 = '''revenue_after ~ C(test_coupon) + C(channel_acq) + minority + non_male +
              num_past_purch + spent_last_purchase + weeks_since_visit +
              browsing_minutes + shopping_cart'''
model2 = smf.ols(formula=formula2, data=data).fit()
model2 = model2.get_robustcov_results(cov_type='HC3')
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:          revenue_after   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1305
Date:                Tue, 18 Nov 2025   Prob (F-statistic):              0.718
Time:                        03:44:18   Log-Likelihood:                -22906.
No. Observations:                5000   AIC:                         4.582e+04
Df Residuals:                    4998   BIC:                         4.583e+04
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               7.7802    



In the uncontrolled regression, the standard error for the coupon effect is 0.669, reflecting the variability in revenue between treatment and control groups without adjusting for customer differences. After controlling for pre-experiment variables like demographics, acquisition channel, and prior behavior, the standard error drops to 0.613. This reduction indicates that the covariates help explain residual variation in revenue, allowing for a more precise estimate of the treatment effect.







In [ ]:
#  Regressions to estimate heterogeneous effects

#A. HTE by Acquisition Channel
formula_hte = 'revenue_after ~ C(test_coupon) * C(channel_acq) + minority + non_male + num_past_purch + spent_last_purchase + weeks_since_visit + browsing_minutes + shopping_cart'
model_hte =smf.ols(formula = formula_hte, data = data).fit()
model_hte = model_hte.get_robustcov_results(cov_type='HC3')
print(model_hte.summary())



#B. HTE by Shopping Cart Status
formula_hte = 'revenue_after ~ C(test_coupon) * C(shopping_cart) + minority + non_male + num_past_purch + spent_last_purchase + weeks_since_visit + browsing_minutes + C(channel_acq)'
model_hte =smf.ols(formula = formula_hte, data = data).fit()
model_hte = model_hte.get_robustcov_results(cov_type='HC3')
print(model_hte.summary())



                            OLS Regression Results                            
Dep. Variable:          revenue_after   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     24.09
Date:                Tue, 18 Nov 2025   Prob (F-statistic):           2.86e-69
Time:                        03:44:18   Log-Likelihood:                -22430.
No. Observations:                5000   AIC:                         4.489e+04
Df Residuals:                    4983   BIC:                         4.501e+04
Df Model:                          16                                         
Covariance Type:                  HC3                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------




There is evidence of heterogeneous treatment effects. In the regression interacting coupon assignment with acquisition channel, the interaction term for channel_acq = 3 is statistically significant (p = 0.037), indicating that the coupon was more effective for customers acquired through this channel compared to others. For shopping cart status, the interaction term was marginally significant (p = 0.062), suggesting that customers with items in their cart may respond more positively to the coupon, though the evidence is not conclusive. These results imply that the impact of the coupon varies across customer subgroups, particularly by acquisition channel.



In [ ]:
# Two other variables for estimating HTEs.

# HTE by num_past_purch

formula_hte = 'revenue_after ~ C(test_coupon) * num_past_purch +minority + non_male + spent_last_purchase + weeks_since_visit + browsing_minutes + C(channel_acq) + shopping_cart'
model_hte =smf.ols(formula = formula_hte, data = data).fit()
model_hte = model_hte.get_robustcov_results(cov_type='HC3')
print(model_hte.summary())


# HTE by weeks_since_visit

formula_hte = 'revenue_after ~ C(test_coupon) * weeks_since_visit + minority + non_male + num_past_purch + spent_last_purchase + browsing_minutes + C(channel_acq) + shopping_cart'
model_hte =smf.ols(formula = formula_hte, data = data).fit()
model_hte = model_hte.get_robustcov_results(cov_type='HC3')
print(model_hte.summary())


                            OLS Regression Results                            
Dep. Variable:          revenue_after   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     29.40
Date:                Tue, 18 Nov 2025   Prob (F-statistic):           8.93e-71
Time:                        03:44:18   Log-Likelihood:                -22421.
No. Observations:                5000   AIC:                         4.487e+04
Df Residuals:                    4986   BIC:                         4.496e+04
Df Model:                          13                                         
Covariance Type:                  HC3                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte


I selected num_past_purch and weeks_since_visit to estimate heterogeneous treatment effects, using interaction terms in separate OLS models. These variables reflect customer purchase history and engagement recency, which could influence responsiveness to the coupon. I found different treatment effects across the two models. The interaction with num_past_purch was statistically significant (p = 0.009), showing that the coupon was less effective for users with more past purchases. This suggests that infrequent buyers are more responsive to promotional offers, while frequent buyers may be less influenced by discounts. In contrast, the interaction with weeks_since_visit was not significant (p = 0.950), indicating no meaningful difference in treatment effect based on how recently a customer visited the site. Overall, these results provide evidence of treatment effect heterogeneity with respect to purchase history, but not with visit recency.


In [ ]:
# Evaluating targeting strategy.

# Simple model: only test_coupon

formula_simple = 'revenue_after ~ test_coupon'
simple_model = smf.ols(data=data, formula=formula_simple).fit()
simple_model = simple_model.get_robustcov_results(cov_type='HC3')

# next, load the data for prediction

## Load New Data and Simulate Targeting Scenarios
predict_data = pd.read_excel('521704-XLS-ENG.xlsx', sheet_name='Next_Campaign_demog')

# We'll look at revenue if we give no one the coupon.
# make a test_coupon column in the new data. Always 0 because no one gets it.
predict_data['test_coupon'] = 0

# find predicted revenue
# note we use the predicted method from model we estimated, which I called
# example_model above.
predict_data['revenue_no_coupon'] = simple_model.predict(predict_data)

# show the output here. It should just be the constant for everyone
print(predict_data['revenue_no_coupon'].head())

total_revenue = predict_data['revenue_no_coupon'].sum()
print('Total revenue with no coupon, example:', total_revenue)

0    7.780168
1    7.780168
2    7.780168
3    7.780168
4    7.780168
Name: revenue_no_coupon, dtype: float64
Total revenue with no coupon, example: 46681.008807045706


In [ ]:
# 5a. Big regression model
formula = (
    'revenue_after ~ C(test_coupon) * (C(channel_acq) + C(shopping_cart) + num_past_purch + weeks_since_visit) + '
    'minority + non_male + spent_last_purchase + browsing_minutes'
)

big_model = smf.ols(formula=formula, data=data).fit()
big_model = big_model.get_robustcov_results(cov_type='HC3')
print(big_model.summary())

                            OLS Regression Results                            
Dep. Variable:          revenue_after   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                     20.66
Date:                Tue, 18 Nov 2025   Prob (F-statistic):           1.26e-68
Time:                        03:44:19   Log-Likelihood:                -22416.
No. Observations:                5000   AIC:                         4.487e+04
Df Residuals:                    4980   BIC:                         4.500e+04
Df Model:                          19                                         
Covariance Type:                  HC3                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------


According to the model summary, the number of coefficients in the model is 20.


In [ ]:
#Predicting future performance

# Dictionary to store total revenue under each strategy
revenue_results = {}

# Everyone gets the coupon
predict_data['test_coupon'] = 1
revenue_results['Everyone gets the coupon'] = model_hte.predict(predict_data).sum()

# No one gets the coupon
predict_data['test_coupon'] = 0
revenue_results['No one gets the coupon'] = model_hte.predict(predict_data).sum()

# Only Instagram users (channel_acq == 3) get the coupon
predict_data['test_coupon'] = np.where(predict_data['channel_acq'] == 3, 1, 0)
revenue_results['instagram_coupon'] = model_hte.predict(predict_data).sum()

# Only users with product in shopping cart get the coupon
predict_data['test_coupon'] = np.where(predict_data['shopping_cart'] == 1, 1,0)
revenue_results['cart_coupon'] = model_hte.predict(predict_data).sum()


# Custom targeting: users with >2 past purchases AND shopping cart

predict_data['test_coupon'] = np.where((predict_data['num_past_purch'] > 2) & (predict_data['shopping_cart'] == 1), 1, 0)
revenue_results['custom_coupon'] = model_hte.predict(predict_data).sum()


#Random 50% targeting (for benchmarking)
np.random.seed(42)
predict_data['test_coupon'] = np.random.binomial(1, 0.5, size=len(predict_data))
revenue_results['random_coupon'] = model_hte.predict(predict_data).sum()


# Print results
for strategy, revenue in revenue_results.items():
    print(f"{strategy:20s}: ${revenue:,.2f}")



Everyone gets the coupon: $44,170.08
No one gets the coupon: $45,834.19
instagram_coupon    : $45,328.39
cart_coupon         : $45,352.44
custom_coupon       : $45,690.66
random_coupon       : $45,005.02



When everone received the coupon , the predicted revenue was $ 44,170.08.

When no one received the coupon, the revenue was $45,834.19.

When only Instagram users got the coupon, the revenue was $45,328.39.

When users with items in their shopping cart got the coupon, the revenue was $45,352.44.

When I targeted users with more than two past purchases and a shopping cart, the revenue was $ 45,690.66


When the coupon was given randomly to 50% of users, the revenue was $45,005.02


## Recommendation
Based on the predicted revenue outcomes from the full model, the most profitable strategy for the next campaign is to not offer the coupon at all, which yields the highest total revenue. However, maximizing short-term revenue may not be the only objective. If the company also aims to re-engage customers, encourage repeat purchases, or strengthen long-term loyalty, then selective coupon targeting can still be valuable. Among the targeted strategies, offering coupons to users with more than two past purchases who also have a product in their shopping cart performs nearly as well as the no-coupon scenario, suggesting it effectively balances revenue generation with customer engagement. Targeting only cart users is another strong option, offering solid revenue with simpler implementation. In contrast, giving the coupon to everyone results in the lowest revenue, indicating that broad, untargeted promotions may not be cost-effective. Therefore, I recommend using a custom targeting strategy that focuses on high-value, high-intent users, as it offers a strong return while supporting broader marketing goals.

